### 监督学习，预测next_move


In [1]:
# 环境配置
%cd /playground/sgd_deep_learning/sgd_rl/go
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rl


In [2]:
from torch.utils.data import DataLoader
from dlgo.data import GoDataset
from dlgo.networks.resnet import resnet34, resnet18
from dlgo.networks.small import CnnSmall

import torch
from torch import nn
import torch.nn.functional as F
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

datatype = torch.float32
# datatype = torch.float16 # 数据容易溢出？？

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device: cuda


### 数据准备
    生成数据，先执行apps/dlgo/data/data_prepare_pipline.ipynb
    然后修改下面的preprocess_dir目录

In [3]:
preprocess_dir = 'data/sevenplane/'
num_workers = 0
train_dataset = GoDataset(preprocess_dir, datatype='train', device=device)
test_dataset = GoDataset(preprocess_dir, datatype='test', device=device)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=num_workers)

print(len(train_dataset), len(test_dataset))
for x,y in train_dataloader:
    print(x.shape, y.shape)
    break

388096 12288
torch.Size([128, 7, 19, 19]) torch.Size([128])


### 模型参数等
    训练集合总共 977920条样本， 测试集合190464条样本
    batch size 64, 0.4s 一个batch的迭代, 一个epoch 需要2h
    batch size 128, 0.9s 一个batch的迭代, 一个epoch 需要2h
    batch size 1024, 15s 一个batch的迭代, 一个epoch 需要4h

    改为resnet18后，
    batch size 128, 0.4s 一个batch的迭代, 一个epoch 需要1h
    进一步改小resnet18
    batch size 128, 0.2s 一个batch的迭代, 一个epoch 需要0.5h
    batch size 256, 0.5s 一个batch的迭代, 一个epoch 需要0.6h

In [4]:
# 不是很好的 chapter-6模型
# cnn_model = nn.Sequential(*[
#     nn.Conv2d(1, 48, kernel_size=3, padding=0, stride=1, bias=False),
#     nn.BatchNorm2d(48),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),
    
#     nn.Conv2d(48, 48, kernel_size=3, padding=0, stride=1, bias=False),
#     nn.BatchNorm2d(48),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2, stride=2,),
#     nn.Dropout(p=0.5),
    
#     nn.Flatten(start_dim=1),
#     nn.Linear(48*7*7, 1024),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),
#     nn.Linear(1024, 19*19),
#     ])

# x = torch.randn(64,1, 19, 19)
# for layer in cnn_model:
#     print(layer)
#     x= layer(x)
#     print(x.shape)

In [5]:
cnn_large_model = nn.Sequential(*[
    nn.Conv2d(7, 64, kernel_size=7, padding=3, stride=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    
    nn.Conv2d(64, 48, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(48),
    nn.ReLU(),
    nn.Conv2d(48, 48, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(48),
    nn.ReLU(),
    
    nn.Conv2d(48, 32, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 32, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    
    nn.Flatten(start_dim=1),
    nn.Linear(32*19*19, 1024),
    nn.ReLU(),
    nn.Linear(1024, 19*19),
    ])

x = torch.randn(64, 7, 19, 19)
for layer in cnn_large_model:
    print(layer)
    x= layer(x)
    print(x.shape)

Conv2d(7, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
torch.Size([64, 64, 19, 19])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 64, 19, 19])
ReLU()
torch.Size([64, 64, 19, 19])
Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
torch.Size([64, 64, 19, 19])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 64, 19, 19])
ReLU()
torch.Size([64, 64, 19, 19])
Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
torch.Size([64, 64, 19, 19])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 64, 19, 19])
ReLU()
torch.Size([64, 64, 19, 19])
Conv2d(64, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
torch.Size([64, 48, 19, 19])
BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 48, 19, 19])
ReLU()
torch.Size([64, 48, 19

In [6]:
cnn_small_model = nn.Sequential(*[
    nn.Conv2d(7, 48, kernel_size=7, padding=3, stride=1, bias=False),
    nn.BatchNorm2d(48),
    nn.ReLU(),
    
    nn.Conv2d(48, 32, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    
    nn.Conv2d(32, 32, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    
    nn.Conv2d(32, 32, kernel_size=5, padding=2, stride=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    
    nn.Flatten(start_dim=1),
    nn.Linear(32*19*19, 512),
    nn.ReLU(),
    nn.Linear(512, 19*19),
    ])

x = torch.randn(64, 7, 19, 19)
for layer in cnn_small_model:
    print(layer)
    x= layer(x)
    print(x.shape)

Conv2d(7, 48, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
torch.Size([64, 48, 19, 19])
BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 48, 19, 19])
ReLU()
torch.Size([64, 48, 19, 19])
Conv2d(48, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
torch.Size([64, 32, 19, 19])
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 32, 19, 19])
ReLU()
torch.Size([64, 32, 19, 19])
Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
torch.Size([64, 32, 19, 19])
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 32, 19, 19])
ReLU()
torch.Size([64, 32, 19, 19])
Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
torch.Size([64, 32, 19, 19])
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([64, 32, 19, 19])
ReLU()
torch.Size([64, 32, 19

In [7]:
def evaluate(model, test_dataloader):
    n_test = len(test_dataloader.dataset)
    with torch.no_grad():
        succ_num = 0
        for x, label in test_dataloader:
            y = model(x)
            # y = torch.randn((x.shape[0], 81), device=device, dtype=datatype) # 测试随机有1.25%的正确率
            # 计算预测正确概率
            # print(type(y), y.shape, y)
            predict = torch.argmax(y, dim=1)
            # print(type(predict), predict.shape, predict)
            # print(type(label),label.shape, label)
            succ_num += torch.sum(predict == label)
        return succ_num/n_test # 预测正确的概率 [瞎猜的概率1/9*9 = 12%]

def train(train_dataloader, model, loss_fn, optimizer, scheduler=None, epochs=1, test_dataloader=None):    
    n_train = len(train_dataloader.dataset) # 总样本量
    best_test_pred = 0
    best_epoch = -1
    
    for epoch in range(epochs):
        acc_loss = 0
        i = 0
        train_num = 0
        train_succ_num = 0
        
        for x, label in train_dataloader:
            y = model(x)
            loss = loss_fn(y, label)
            # print(y.shape,y)
            # print(loss.shape, loss.item())
            # break
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # 预测数据统计
            train_succ_num += torch.sum(torch.argmax(y, dim=1) == label)
            train_num += x.shape[0]
            acc_loss += loss.item()
            i += 1
            
            # batch日志输出
            if i%200 == 0:
                print("epoch:{}, i:{}/{}, avg_loss:{:.3f}, avg_succ{:.3f}%"
                      .format(epoch, i,
                              n_train//x.shape[0],
                              acc_loss/train_num,
                              train_succ_num / train_num * 100,
                              ))
        if scheduler: # 每个epoch更新一次
            scheduler.step()
        #########  end epoch  ###########
        print("-----------------------------")
        # 训练集的准确率
        print("Epoch {}: train_pred_succ:{:.3f}%, {}".format(epoch, train_succ_num / train_num * 100, train_num))
        
        if test_dataloader:
            n_test = len(test_dataloader.dataset) 
            test_succ_ratio = evaluate(model, test_dataloader) * 100
            
            if test_succ_ratio > best_test_pred:
                best_test_pred = test_succ_ratio
                best_epoch = epoch 
            
            print("Epoch {}: test_pred_succ:{:.3f}%, {}".format(epoch, test_succ_ratio, n_test))
        else:
            print("Epoch {0} complete".format(epoch))
            
        print('\n')

    # 训练结束
    print("best_epoch:{}, best_test_pred:{:.3f}%".format(best_epoch, best_test_pred))

### 训练记录

200局游戏，100局验证  best_test_pred:1.839%， 迭代了几十轮
2000局游戏，100局验证 ，5轮epoch

In [8]:
# model = resnet18(num_classes=19*19, encoder_planes=7).to(datatype).to(device)
# model = cnn_model.to(datatype).to(device)
# model = cnn_small_model.to(datatype).to(device)
model = cnn_large_model.to(datatype).to(device)
ce_loss = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# sgd = SGD(lr=0.1, momentum=0.9, decay=0.01) 每次迭代，lr*0.99

# 两个不用调lr的优化器
# optimizer = torch.optim.Adagrad(model.parameters()) # 还不错
# optimizer = torch.optim.Adadelta(model.parameters()) # 也还行

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# StepLR 调度器每隔 `step_size` 个 epoch 将学习率乘以 `gamma`

t1 = time.time()
train(model=model,
      epochs=5,
      loss_fn=ce_loss,
      optimizer=optimizer,
      scheduler=scheduler,
      train_dataloader=train_dataloader,
      test_dataloader=test_dataloader,)
print('{:.3f}s'.format(time.time()-t1))

epoch:0, i:200/3032, avg_loss:0.045, avg_succ0.410%
epoch:0, i:400/3032, avg_loss:0.045, avg_succ0.479%
epoch:0, i:600/3032, avg_loss:0.045, avg_succ0.561%
epoch:0, i:800/3032, avg_loss:0.045, avg_succ0.691%
epoch:0, i:1000/3032, avg_loss:0.044, avg_succ0.784%
epoch:0, i:1200/3032, avg_loss:0.044, avg_succ0.929%
epoch:0, i:1400/3032, avg_loss:0.044, avg_succ1.053%
epoch:0, i:1600/3032, avg_loss:0.043, avg_succ1.176%
epoch:0, i:1800/3032, avg_loss:0.043, avg_succ1.309%
epoch:0, i:2000/3032, avg_loss:0.043, avg_succ1.446%
epoch:0, i:2200/3032, avg_loss:0.042, avg_succ1.589%
epoch:0, i:2400/3032, avg_loss:0.042, avg_succ1.714%
epoch:0, i:2600/3032, avg_loss:0.042, avg_succ1.860%
epoch:0, i:2800/3032, avg_loss:0.042, avg_succ1.982%
epoch:0, i:3000/3032, avg_loss:0.042, avg_succ2.105%
-----------------------------
Epoch 0: train_pred_succ:2.118%, 388096
Epoch 0: test_pred_succ:4.069%, 12288


epoch:1, i:200/3032, avg_loss:0.039, avg_succ4.234%
epoch:1, i:400/3032, avg_loss:0.038, avg_succ4.

chapter_6 cnn架构

    Epoch 4: train_pred_succ:2.424%, 388096
    Epoch 4: test_pred_succ:2.547%, 12288
    best_epoch:4, best_test_pred:2.547%
    669.404s


chapter_7 small cnn 架构

    * 训练集2000
    Epoch 4: train_pred_succ:19.276%, 388096
    Epoch 4: test_pred_succ:18.530%, 12288
    best_epoch:4, best_test_pred:18.530%
    556.572s
    结论:多cnn结构，效果马上好起来了

    * 加大数据量和epoch数
        训练集10000，epoch-10

        Epoch 3: train_pred_succ:29.281%, 1961984
        Epoch 3: test_pred_succ:24.227%, 12288
        不知道极限是多少，但是训练过程很慢，效果还在缓慢变好

7层encoder-2000games-small_cnn架构
    
    效果还不错，每个epoch都能涨几个点，但是过5遍epoch完全不够。
    Epoch 4: train_pred_succ:33.546%, 388096
    Epoch 4: test_pred_succ:22.534%, 12288
    best_epoch:4, best_test_pred:22.534%
    1121.978s

7层encoder-2000games-resnet18架构

    确实要优于普通cnn，同样计算代价下收敛更快，且不容易过拟合
    Epoch 4: train_pred_succ:38.685%, 388096
    Epoch 4: test_pred_succ:32.886%, 12288
    best_epoch:4, best_test_pred:32.886%
    1022.198s

7层encoder-2000games-large_cnn架构
    
    如果三轮到不了书上的效果，就是学习率和数据量的问题
    速度方面也还行，不算太慢

    Epoch 4: train_pred_succ:35.159%, 388096
    poch 4: test_pred_succ:24.365%, 12288
    best_epoch:4, best_test_pred:24.365%
    1004.615s
    是不是线性层没加dropout，不如resnet那个
    
    




## playground

In [10]:
# target output size of 5x7
m = nn.AdaptiveAvgPool2d((5, 7))
input = torch.randn(1, 64, 8, 9)
output = m(input)
print(input.shape, output.shape)

# target output size of 7x7 (square)
m = nn.AdaptiveAvgPool2d(7)
input = torch.randn(1, 64, 10, 9)
output = m(input)
print(input.shape, output.shape)

# target output size of 10x7
m = nn.AdaptiveAvgPool2d((None, 7))
input = torch.randn(1, 64, 10, 9)
output = m(input)
print(input.shape, output.shape)

torch.Size([1, 64, 8, 9]) torch.Size([1, 64, 5, 7])
torch.Size([1, 64, 10, 9]) torch.Size([1, 64, 7, 7])
torch.Size([1, 64, 10, 9]) torch.Size([1, 64, 10, 7])


### cnnsmall
    log1 torch.Size([1, 48, 17, 17])
    log2 torch.Size([1, 32, 15, 15])
    log3 torch.Size([1, 32, 13, 13])
    log4 torch.Size([1, 32, 11, 11])
    log5-flatten torch.Size([1, 3872])
    log6-fc1 torch.Size([1, 722])
    log7-fc3 torch.Size([1, 361])
    torch.Size([1, 361])

In [11]:
# 测试模型
model = CnnSmall(encoder_planes=1)
# print(model)

input_data = torch.randn(64, 1, 19, 19)  # 假设输入为 3 通道，224x224 大小的图像
output = model(input_data)
print(output.size()) # expansion 直接就是分类模型了

torch.Size([64, 361])


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
